# Phishing Detection Training

- This notebook will implement and test multiple models for URL detection

First we import libraries and get data. We will split the data into:

Training - 70%

Cross Validation - 10%

Testing - 20%

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import evaluation_functions as evaluate

In [4]:
data = pd.read_csv('../data/data_featured.csv', index_col=0)
print("Columns Loaded in: ", list(data.columns))

X = data.drop('Label', axis=1).drop("URL", axis=1)
Y = data['Label']

# Split into training and testing set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

# Split training so we get 10% overrall cross validation
x_train, x_cross, y_train, y_cross = train_test_split(x_train, y_train,
                                                  test_size=0.125, random_state=1)

Columns Loaded in:  ['URL', 'Label', 'periods', 'slashes', 'length', 'numberCount', 'firstdigit', 'IP', 'semicolon', 'domain', 'hash']
Row
0         1
1         1
2         1
3         1
4         1
         ..
549341    1
549342    1
549343    1
549344    1
549345    1
Name: Label, Length: 549346, dtype: int64


# Train

Now we train a simple Random Forest classifier. We will evaluate using accuracies, f-score and confusion matricies. Also we can output the incorrect false negatives and positives into a .csv file to find out what we are doing wrong.

In [3]:


print('Training Model - Random Forest ...')

forest = RandomForestClassifier(min_samples_leaf=3)
forest.fit(x_train, y_train)

print("Model Trained - Random Forest ... ")

train_predicts, cross_predicts, test_predicts = evaluate.print_accuracies(forest, x_train, y_train, x_cross, y_cross, x_test, y_test)
evaluate.confusion_matrix_plot(y_test, test_predicts, title='Random Forest Confusion Matrix')

Training Model - Random Forest ...


ValueError: could not convert string to float: 'appleimagecache.com/'

In [ ]:
true_positives = y_cross == 1
predicted_negatives = cross_predicts == 0

og_data = pd.read_csv('../data/phishing_site_urls.csv')

false_negative_indexes = np.array((true_positives != predicted_negatives)).nonzero()[0]

og_data.iloc[false_negative_indexes, 0].to_csv("../data/false_negatives_cross_validation.csv")

In [ ]:
import pickle

with open('my_dumped_classifier.pkl', 'wb') as fid:
    pickle.dump(forest, fid)